## Скачиваем необходимое

Сначала нужно средствами NLTK загрузить WordNet.



In [1]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

## Готовим данные к работе

Затем импортируем данные из подготовленного текстового файла. Файл содержит набор пар слов (только имён существительных), для которых известны экспертные оценки сходства.

Строим ассоциативный массив "пара слов -- оценка близости".

In [16]:
import pandas as pd

df = pd.read_csv('https://courses.openedu.ru/assets/courseware/v1/e585af30b55a89a85efcabc6a6ac0bfa/asset-v1:ITMOUniversity+INFRETTECH+fall_2024_ITMO_bac+type@asset+block/Task_4_sample_4.csv')
score_map = {tuple(df.iloc[i, :2]): float(df.iloc[i, 2]) for i in range(len(df))}

Вычисляем для всех пар несколько оценок

In [17]:
from itertools import product

list_pairs = list(score_map)
wup_list, true_list, path_list, lch_list = [], [], [], []

# для всех пар
for w1, w2 in list_pairs:

  try:
    all_w1 = wn.synsets(w1, pos="n")
    all_w2 = wn.synsets(w2, pos="n")

    # добавляем интересующие нас метрики и экспертные оценки
    wup = max([item1.wup_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    wup_list.append(wup)

    path = max([item1.path_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    path_list.append(path)

    lch = max([item1.lch_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    lch_list.append(lch)

    true_list.append(score_map[(w1, w2)])

  except Exception as e:
    print(w1, w2, "error:", e)

## Вычисляем ранговую корреляцию Спирмена

In [19]:
from scipy.stats import spearmanr

coef, p = spearmanr(wup_list, true_list)
print("wup  Spearman R: %.3f" % coef)

coef, p = spearmanr(path_list, true_list)
print("path Spearman R: %.3f" % coef)

coef, p = spearmanr(lch_list, true_list)
print("lch Spearman R: %.3f" % coef)

wup  Spearman R: 0.694
path Spearman R: 0.654
lch Spearman R: 0.654




---

При помощи метода `hyponyms()` найдите количество гипонимов для синсета `wood.n.01`, а также при помощи метода `name()` найдите значение первого в гипонима из списка.

In [20]:
synset = wn.synset('wood.n.01')
hyponyms = synset.hyponyms()

num_hyponyms = len(hyponyms)
print(f'Количество гипонимов для wood.n.01: {num_hyponyms}')

Количество гипонимов для wood.n.01: 91


In [21]:
first_hyponym_name = hyponyms[0].name()
print(f'Первый гипоним: {first_hyponym_name}')

Первый гипоним: alder.n.01
